In [ ]:
import json
import os
import pandas as pd
import numpy as np

In [15]:
import warnings
warnings.filterwarnings('ignore')

In [16]:
SEED = 42
DROP_APPROACHES = [
    'BinSeg Clinear',
]
EXPECTED_NUMBER_APPROACHES = (3 * 10) + 1 - len(DROP_APPROACHES)

In [17]:
def get_df_for_dataset(database_name: str, dataset_name: str) -> pd.DataFrame:
    def get_df_for_dataset_inner(database_name: str, dataset_name: str) -> pd.DataFrame:
        directory_path = f"outputs/report/seed={SEED}/{database_name}/{dataset_name}"

        print(f"Reading from {directory_path}")

        metrics_dfs = []
        
        for root, dirs, files in os.walk(directory_path):
            for filename in files:
                if filename.endswith('.json'):
                    file_path = os.path.join(root, filename)
                    
                    try:
                        with open(file_path, 'r') as file:
                            data = json.load(file)
                            metrics_dfs.append(
                                pd.DataFrame({
                                    'cut_point_model': [data['cut_point_model']],
                                    'cut_point_method': [data['cut_point_method']],
                                    'cut_point_approach': [data['cut_point_approach']],
                                    'cut_point_perc': [data['cut_point_perc']],
                                    'cut_duration': [data['cut_duration']],
                                    'cut_duration_minutes': [data['cut_duration'] / 60],
                                    'tuner_duration': [data['tuner_duration']],
                                    'tuner_duration_minutes': [data['tuner_duration'] / 60],
                                    'retrain_duration': [data['retrain_duration']],
                                    'retrain_duration_minutes': [data['retrain_duration'] / 60],
                                    'total_duration': [data['total_duration']],
                                    'total_duration_minutes': [data['total_duration'] / 60],
                                    'Avg_MAPE': [data['error_results']['Avg_MAPE']],
                                    'Avg_MAE': [data['error_results']['Avg_MAE']], 
                                    'Avg_MSE': [data['error_results']['Avg_MSE']], 
                                    'Avg_RMSE': [data['error_results']['Avg_RMSE']], 
                                    'Avg_R2':  [data['error_results']['Avg_R2']],
                                    'Avg_WAPE':  [data['error_results']['Avg_WAPE']],
                                }))
                    except KeyError as e:
                        print(f"Error in {file_path}: {e}")
                        continue
                    
        metrics_df = pd.concat(metrics_dfs, axis=0)
    
        metrics_df['cut_point_approach'] = metrics_df['cut_point_approach'].str.replace('Rbf', 'RBF')
        metrics_df['cut_point_approach'] = metrics_df['cut_point_approach'].str.replace(' Ar', ' AR')
        metrics_df['cut_point_approach'] = metrics_df['cut_point_approach'].str.replace('Bin_Seg', 'BinSeg')
        metrics_df['cut_point_approach'] = metrics_df['cut_point_approach'].str.replace('Bottom_Up', 'BottomUp')
        metrics_df['cut_point_approach'] = metrics_df['cut_point_approach'].str.replace('Fixed_Perc Fixed_Cut_0.0', 'Full')
        metrics_df['cut_point_approach'] = metrics_df['cut_point_approach'].str.replace('Fixed_Perc Fixed_Cut_', 'Fixed Cut ')
        metrics_df['cut_point_approach'] = metrics_df['cut_point_approach'].str.replace('0.1', '10%')
        metrics_df['cut_point_approach'] = metrics_df['cut_point_approach'].str.replace('0.2', '20%')
        metrics_df['cut_point_approach'] = metrics_df['cut_point_approach'].str.replace('0.3', '30%')
        metrics_df['cut_point_approach'] = metrics_df['cut_point_approach'].str.replace('0.4', '40%')
        metrics_df['cut_point_approach'] = metrics_df['cut_point_approach'].str.replace('0.5', '50%')
        metrics_df['cut_point_approach'] = metrics_df['cut_point_approach'].str.replace('0.6', '60%')
        metrics_df['cut_point_approach'] = metrics_df['cut_point_approach'].str.replace('0.7', '70%')
        metrics_df['cut_point_approach'] = metrics_df['cut_point_approach'].str.replace('0.8', '80%')

        metrics_df = metrics_df[~metrics_df['cut_point_approach'].str.endswith(tuple(DROP_APPROACHES))]

        return metrics_df

    metrics_df = get_df_for_dataset_inner(
        database_name, dataset_name
    )

    metrics_cols = metrics_df.columns.tolist()
    metrics_df['database'] = [database_name] * len(metrics_df)
    metrics_df['dataset'] = [dataset_name] * len(metrics_df)
    metrics_df = metrics_df[['database', 'dataset'] + metrics_cols]

    return metrics_df.reset_index(drop=True)

In [18]:
datasets = []

# [UCI] APPLIANCES_ENERGY

In [19]:
uci_appliances_energy = get_df_for_dataset('UCI', 'APPLIANCES_ENERGY')
uci_appliances_energy_name = 'UCI APPLIANCES_ENERGY'
assert not uci_appliances_energy['cut_point_approach'].duplicated().any(), "Duplicate values found in column 'cut_point_approach'"
assert len(uci_appliances_energy) == EXPECTED_NUMBER_APPROACHES, f"Expected {EXPECTED_NUMBER_APPROACHES} lines but got {len(uci_appliances_energy)}"

datasets.append(uci_appliances_energy)

uci_appliances_energy.sort_values(by='Avg_RMSE')

Reading from outputs/report/seed=42/UCI/APPLIANCES_ENERGY


,database,dataset,cut_point_model,cut_point_method,cut_point_approach,cut_point_perc,cut_duration,cut_duration_minutes,tuner_duration,tuner_duration_minutes,retrain_duration,retrain_duration_minutes,total_duration,total_duration_minutes,Avg_MAPE,Avg_MAE,Avg_MSE,Avg_RMSE,Avg_R2,Avg_WAPE
3,UCI,APPLIANCES_ENERGY,Bottom_Up,RBF,BottomUp RBF,61.787433,3.430994,5.718324e-02,625.520150,10.425336,1.677591,0.027960,630.628735,10.510479,1.016629e+12,2.053687,10.852670,2.746970,0.822843,0.013861
5,UCI,APPLIANCES_ENERGY,Bottom_Up,Normal,BottomUp Normal,63.624272,0.094778,1.579630e-03,640.668585,10.677810,1.653634,0.027561,642.416997,10.706950,1.180656e+12,2.099043,11.438883,2.782456,0.825742,0.014167
27,UCI,APPLIANCES_ENERGY,Bin_Seg,L1,BinSeg L1,68.184697,2.228780,3.714633e-02,563.319168,9.388653,1.615926,0.026932,567.163874,9.452731,1.338553e+12,2.131524,11.497177,2.799424,0.817620,0.014386
22,UCI,APPLIANCES_ENERGY,Bin_Seg,L2,BinSeg L2,51.779833,0.612371,1.020619e-02,1143.538249,19.058971,2.467542,0.041126,1146.618162,19.110303,1.706733e+12,2.115571,11.900477,2.804179,0.817813,0.014279
20,UCI,APPLIANCES_ENERGY,Bin_Seg,Cosine,BinSeg Cosine,51.779833,238.927329,3.982122e+00,1070.205597,17.836760,2.248501,0.037475,1311.381427,21.856357,1.706733e+12,2.115571,11.900477,2.804179,0.817813,0.014279
26,UCI,APPLIANCES_ENERGY,Bin_Seg,Mahalanobis,BinSeg Mahalanobis,61.850773,239.521604,3.992027e+00,635.467338,10.591122,1.745147,0.029086,876.734089,14.612235,1.020582e+12,2.106785,11.294061,2.804445,0.813098,0.014219
0,UCI,APPLIANCES_ENERGY,Bottom_Up,Cosine,BottomUp Cosine,41.740562,1.289272,2.148786e-02,1248.430125,20.807169,1.615190,0.026920,1251.334587,20.855576,1.594236e+12,2.147750,11.875874,2.808984,0.814846,0.014496
28,UCI,APPLIANCES_ENERGY,Bin_Seg,Rank,BinSeg Rank,62.674183,0.571174,9.519569e-03,734.281951,12.238033,1.610975,0.026850,736.464100,12.274402,1.359677e+12,2.121773,11.901508,2.813785,0.824566,0.014320
23,UCI,APPLIANCES_ENERGY,Bin_Seg,RBF,BinSeg RBF,67.329617,261.164436,4.352741e+00,554.296470,9.238275,1.561676,0.026028,817.022582,13.617043,1.485707e+12,2.124878,12.359323,2.826999,0.823621,0.014341
24,UCI,APPLIANCES_ENERGY,Bin_Seg,AR,BinSeg AR,38.985305,3.501432,5.835720e-02,1193.216145,19.886936,1.444882,0.024081,1198.162459,19.969374,1.272222e+12,2.138227,11.677589,2.827241,0.819468,0.014432


# [UCI] METRO_TRAFFIC

In [20]:
uci_metro_traffic = get_df_for_dataset('UCI', 'METRO_TRAFFIC')
uci_metro_traffic_name = 'UCI METRO_TRAFFIC'
assert not uci_metro_traffic['cut_point_approach'].duplicated().any(), "Duplicate values found in column 'cut_point_approach'"
assert len(uci_metro_traffic) == EXPECTED_NUMBER_APPROACHES, f"Expected {EXPECTED_NUMBER_APPROACHES} lines but got {len(uci_metro_traffic)}"

datasets.append(uci_metro_traffic)

uci_metro_traffic.sort_values(by='Avg_RMSE')

Reading from outputs/report/seed=42/UCI/METRO_TRAFFIC


,database,dataset,cut_point_model,cut_point_method,cut_point_approach,cut_point_perc,cut_duration,cut_duration_minutes,tuner_duration,tuner_duration_minutes,retrain_duration,retrain_duration_minutes,total_duration,total_duration_minutes,Avg_MAPE,Avg_MAE,Avg_MSE,Avg_RMSE,Avg_R2,Avg_WAPE
17,UCI,METRO_TRAFFIC,Window,L1,Window L1,78.404170,0.306833,5.113880e-03,796.321296,13.272022,4.449573,0.074160,801.077702,13.351295,1.301584e+14,9.919907,319.205319,12.268973,0.400739,0.090789
13,UCI,METRO_TRAFFIC,Window,RBF,Window RBF,78.404170,163.603737,2.726729e+00,739.170359,12.319506,4.256316,0.070939,907.030412,15.117174,1.301584e+14,9.919907,319.205319,12.268973,0.400739,0.090789
16,UCI,METRO_TRAFFIC,Window,Mahalanobis,Window Mahalanobis,64.543734,10.135399,1.689233e-01,815.381512,13.589692,10.357770,0.172629,835.874681,13.931245,1.738452e+14,10.009012,328.524894,12.471191,0.496144,0.091604
12,UCI,METRO_TRAFFIC,Window,L2,Window L2,64.543734,0.198700,3.311666e-03,862.014581,14.366910,11.325171,0.188753,873.538452,14.558974,1.738452e+14,10.009012,328.524894,12.471191,0.496144,0.091604
14,UCI,METRO_TRAFFIC,Window,AR,Window AR,64.504836,0.240266,4.004435e-03,768.708243,12.811804,1.929315,0.032155,770.877824,12.847964,1.183824e+14,10.554436,338.733748,12.949444,0.499897,0.096596
27,UCI,METRO_TRAFFIC,Bin_Seg,L1,BinSeg L1,14.197547,8.506616,1.417769e-01,2111.504704,35.191745,4.431029,0.073850,2124.442349,35.407372,3.475975e+15,10.755650,325.358834,12.969699,-9.714579,0.098438
23,UCI,METRO_TRAFFIC,Bin_Seg,RBF,BinSeg RBF,14.197547,4411.513318,7.352522e+01,2129.361428,35.489357,4.621214,0.077020,6545.495960,109.091599,3.475975e+15,10.755650,325.358834,12.969699,-9.714579,0.098438
22,UCI,METRO_TRAFFIC,Bin_Seg,L2,BinSeg L2,14.197547,1.456047,2.426745e-02,2066.999711,34.449995,4.269035,0.071151,2072.724793,34.545413,3.475975e+15,10.755650,325.358834,12.969699,-9.714579,0.098438
10,UCI,METRO_TRAFFIC,Window,Cosine,Window Cosine,6.923735,26.018665,4.336444e-01,2251.353688,37.522561,153.205313,2.553422,2430.577666,40.509628,9.518896e+14,11.305071,338.781476,13.369726,-0.211067,0.103466
11,UCI,METRO_TRAFFIC,Window,Linear,Window Linear,30.962321,0.171714,2.861901e-03,1417.825688,23.630428,53.588565,0.893143,1471.585967,24.526433,1.512592e+15,11.268606,335.823715,13.394868,-1.684863,0.103132


# [UCI] PRSA_BEIJING

In [21]:
uci_prsa_beijing = get_df_for_dataset('UCI', 'PRSA_BEIJING')
uci_prsa_beijing_name = 'UCI PRSA_BEIJING'
assert not uci_prsa_beijing['cut_point_approach'].duplicated().any(), "Duplicate values found in column 'cut_point_approach'"
assert len(uci_prsa_beijing) == EXPECTED_NUMBER_APPROACHES, f"Expected {EXPECTED_NUMBER_APPROACHES} lines but got {len(uci_prsa_beijing)}"

datasets.append(uci_prsa_beijing)

uci_prsa_beijing.sort_values(by='Avg_RMSE')

Reading from outputs/report/seed=42/UCI/PRSA_BEIJING


,database,dataset,cut_point_model,cut_point_method,cut_point_approach,cut_point_perc,cut_duration,cut_duration_minutes,tuner_duration,tuner_duration_minutes,retrain_duration,retrain_duration_minutes,total_duration,total_duration_minutes,Avg_MAPE,Avg_MAE,Avg_MSE,Avg_RMSE,Avg_R2,Avg_WAPE
19,UCI,PRSA_BEIJING,Window,Clinear,Window Clinear,25.143330,0.148353,2.472552e-03,1580.747247,26.345787,12.665811,0.211097,1593.561411,26.559357,3.026122e+14,3.605650,106.714078,6.117033,0.705697,0.020338
0,UCI,PRSA_BEIJING,Bottom_Up,Cosine,BottomUp Cosine,25.000713,23.862448,3.977075e-01,1731.530996,28.858850,12.626196,0.210437,1768.019640,29.466994,2.201594e+14,3.754412,109.206367,6.193911,0.735155,0.021177
2,UCI,PRSA_BEIJING,Bottom_Up,L2,BottomUp L2,25.000713,0.123301,2.055017e-03,1707.137046,28.452284,13.668828,0.227814,1720.929175,28.682153,2.201594e+14,3.754412,109.206367,6.193911,0.735155,0.021177
17,UCI,PRSA_BEIJING,Window,L1,Window L1,25.000713,0.305785,5.096420e-03,1820.568938,30.342816,13.356126,0.222602,1834.230849,30.570514,2.201594e+14,3.754412,109.206367,6.193911,0.735155,0.021177
12,UCI,PRSA_BEIJING,Window,L2,Window L2,25.000713,0.188871,3.147848e-03,1785.857577,29.764293,13.382681,0.223045,1799.429129,29.990485,2.201594e+14,3.754412,109.206367,6.193911,0.735155,0.021177
10,UCI,PRSA_BEIJING,Window,Cosine,Window Cosine,25.000713,24.075896,4.012649e-01,1758.345270,29.305754,12.695156,0.211586,1795.116322,29.918605,2.201594e+14,3.754412,109.206367,6.193911,0.735155,0.021177
16,UCI,PRSA_BEIJING,Window,Mahalanobis,Window Mahalanobis,0.199663,8.374054,1.395676e-01,2274.842819,37.914047,4.314529,0.071909,2287.531402,38.125523,2.595686e+14,3.965704,117.432993,6.452136,0.703868,0.022368
18,UCI,PRSA_BEIJING,Window,Rank,Window Rank,0.213925,0.196797,3.279952e-03,2010.067389,33.501123,4.188887,0.069815,2014.453073,33.574218,2.704385e+14,4.058416,118.769662,6.488261,0.703433,0.022891
27,UCI,PRSA_BEIJING,Bin_Seg,L1,BinSeg L1,5.989903,13.866350,2.311058e-01,1886.902319,31.448372,3.488268,0.058138,1904.256937,31.737616,2.013889e+14,4.087950,120.050562,6.491163,0.718550,0.023058
23,UCI,PRSA_BEIJING,Bin_Seg,RBF,BinSeg RBF,5.975641,2978.020171,4.963367e+01,1861.717439,31.028624,4.074171,0.067903,4843.811781,80.730196,1.988101e+14,4.027712,118.710691,6.522623,0.703423,0.022718


# [UCI] AIR_QUALITY

In [22]:
uci_air_quality = get_df_for_dataset('UCI', 'AIR_QUALITY')
uci_air_quality_name = 'UCI AIR_QUALITY'
assert not uci_air_quality['cut_point_approach'].duplicated().any(), "Duplicate values found in column 'cut_point_approach'"
assert len(uci_air_quality) == EXPECTED_NUMBER_APPROACHES, f"Expected {EXPECTED_NUMBER_APPROACHES} lines but got {len(uci_air_quality)}"

datasets.append(uci_air_quality)

uci_air_quality.sort_values(by='Avg_RMSE')

Reading from outputs/report/seed=42/UCI/AIR_QUALITY
Error in outputs/report/seed=42/UCI/AIR_QUALITY/Bin_Seg/Clinear/2025-02-28_03-51-56/report.json: 'tuner_duration'
Error in outputs/report/seed=42/UCI/AIR_QUALITY/Bin_Seg/Clinear/2025-03-05_23-00-53/report.json: 'tuner_duration'


,database,dataset,cut_point_model,cut_point_method,cut_point_approach,cut_point_perc,cut_duration,cut_duration_minutes,tuner_duration,tuner_duration_minutes,retrain_duration,retrain_duration_minutes,total_duration,total_duration_minutes,Avg_MAPE,Avg_MAE,Avg_MSE,Avg_RMSE,Avg_R2,Avg_WAPE
10,UCI,AIR_QUALITY,Window,Cosine,Window Cosine,10.754843,0.213670,3.561167e-03,247.522309,4.125372,5.279954,0.087999,253.015933,4.216932,5.092353e+12,26.342861,4391.055889,36.431655,0.447287,0.308531
24,UCI,AIR_QUALITY,Bin_Seg,AR,BinSeg AR,56.312625,0.846187,1.410311e-02,267.195344,4.453256,3.011614,0.050194,271.053145,4.517552,3.534100e+12,29.777870,5285.321990,39.378808,0.453840,0.348763
22,UCI,AIR_QUALITY,Bin_Seg,L2,BinSeg L2,56.312625,0.149547,2.492452e-03,268.464293,4.474405,3.108406,0.051807,271.722246,4.528704,3.534100e+12,29.777870,5285.321990,39.378808,0.453840,0.348763
4,UCI,AIR_QUALITY,Bottom_Up,AR,BottomUp AR,55.778223,0.036219,6.036480e-04,178.489141,2.974819,1.555611,0.025927,180.080971,3.001350,4.411839e+12,31.735471,5675.028011,41.373853,0.305369,0.371690
1,UCI,AIR_QUALITY,Bottom_Up,Linear,BottomUp Linear,51.102204,0.030406,5.067666e-04,159.534918,2.658915,1.530504,0.025508,161.095828,2.684930,4.914077e+12,32.796998,5699.635776,41.406372,0.265486,0.384123
0,UCI,AIR_QUALITY,Bottom_Up,Cosine,BottomUp Cosine,54.976620,0.306578,5.109632e-03,156.459394,2.607657,1.592681,0.026545,158.358653,2.639311,4.089651e+12,32.364435,5821.314658,41.481614,0.363526,0.379057
26,UCI,AIR_QUALITY,Bin_Seg,Mahalanobis,BinSeg Mahalanobis,65.531062,31.867070,5.311178e-01,168.314000,2.805233,2.536734,0.042279,202.717804,3.378630,2.585569e+12,31.573567,5851.571096,41.654272,0.425213,0.369794
21,UCI,AIR_QUALITY,Bin_Seg,Linear,BinSeg Linear,52.371409,0.477966,7.966101e-03,160.086341,2.668106,1.592283,0.026538,162.156590,2.702610,4.337728e+12,32.372975,5751.198019,41.742779,0.294036,0.379157
29,UCI,AIR_QUALITY,Fixed_Perc,Fixed_Cut_0.0,Full,0.000000,0.000007,1.231829e-07,250.893080,4.181551,1.475167,0.024586,252.368254,4.206138,4.862309e+12,28.869095,5670.115617,41.809212,0.219392,0.338119
9,UCI,AIR_QUALITY,Bottom_Up,Clinear,BottomUp Clinear,56.513026,0.022140,3.690004e-04,197.753316,3.295889,1.679677,0.027995,199.455133,3.324252,3.911079e+12,33.296230,6008.218288,41.961072,0.363048,0.389970


# [INMET] SAOPAULO_SP

In [23]:
# inmet_saopaulo_sp = get_df_for_dataset('INMET', 'SAOPAULO_SP')
# inmet_saopaulo_sp_name = 'INMET SAOPAULO_SP'
# assert not inmet_saopaulo_sp['cut_point_approach'].duplicated().any(), "Duplicate values found in column 'cut_point_approach'"
# assert len(inmet_saopaulo_sp) == EXPECTED_NUMBER_APPROACHES, f"Expected {EXPECTED_NUMBER_APPROACHES} lines but got {len(inmet_saopaulo_sp)}"

# datasets.append(inmet_saopaulo_sp)

# inmet_saopaulo_sp.sort_values(by='Avg_RMSE')

# Rank

In [24]:
def compute_arr(df, acc_d, algorithm_p, algorithm_q, dataset, metric):
    df_d = df[df['dataset_name'] == dataset]

    d_ap = df_d[df_d["cut_point_approach"] == algorithm_p]
    d_aq = df_d[df_d["cut_point_approach"] == algorithm_q]

    assert len(d_ap) == 1, f"Expected 1 line for approach {algorithm_p} but got {len(d_ap)}"
    assert len(d_aq) == 1, f"Expected 1 line for approach {algorithm_q} but got {len(d_aq)}"

    sr_ap = 1 / d_ap[metric].iloc[0]
    sr_aq = 1 / d_aq[metric].iloc[0]

    t_ap = d_ap["total_duration"].iloc[0]
    t_aq = d_aq["total_duration"].iloc[0]

    sr_ratio = sr_ap / sr_aq if sr_aq != 0 else np.nan  # Avoid division by zero
    time_ratio = np.log(t_ap / t_aq) if t_aq != 0 else np.nan  # Avoid division by zero

    arr = sr_ratio / (1 + acc_d * time_ratio)
    
    return arr


def compute_arr_ranking(df, acc_d, metric):
    algorithms = df["cut_point_approach"].unique()
    datasets = df['dataset_name'].unique()
    arr_df = pd.DataFrame({
        'dataset': [],
        'ap': [],
        'aq': [],
        'arr': [],
    })

    for dataset in datasets:
        for ap in algorithms:
            for aq in algorithms:
                if ap == aq:
                    continue

                arr_df = pd.concat([arr_df, pd.DataFrame({
                    'dataset': [dataset],
                    'ap': [ap],
                    'aq': [aq],
                    'arr': [compute_arr(df, acc_d, ap, aq, dataset, metric)],
                })])

    grouped = arr_df.groupby(['ap', 'aq'])['arr'].apply(lambda x: np.prod(x) ** (1 / len(x))).reset_index()

    ranking_df = grouped.groupby('ap')['arr'].mean().reset_index()
    ranking_df['rank'] = ranking_df['arr'].rank(ascending=False, method='max')

    return ranking_df.sort_values(by='rank')

In [25]:
metrics_df = pd.concat(datasets, axis=0)
metrics_df = metrics_df[~metrics_df['cut_point_approach'].isin(DROP_APPROACHES)]
metrics_df['dataset_name'] = metrics_df['database'] + ' ' + metrics_df['dataset']

## WAPE

In [26]:
arr_ranking_df = compute_arr_ranking(metrics_df, acc_d=0.05, metric='Avg_WAPE')
arr_ranking_df

,ap,arr,rank
22,Window Cosine,1.138345,1.0
23,Window L1,1.115929,2.0
2,BinSeg L1,1.102061,3.0
3,BinSeg L2,1.100280,4.0
24,Window L2,1.096968,5.0
13,BottomUp L2,1.094207,6.0
11,BottomUp Cosine,1.086910,7.0
28,Window RBF,1.080698,8.0
5,BinSeg Mahalanobis,1.072866,9.0
7,BinSeg RBF,1.072298,10.0


In [27]:
arr_ranking_df = compute_arr_ranking(metrics_df, acc_d=0.1, metric='Avg_WAPE')
arr_ranking_df

,ap,arr,rank
22,Window Cosine,1.120930,1.0
23,Window L1,1.111084,2.0
13,BottomUp L2,1.099076,3.0
2,BinSeg L1,1.097277,4.0
24,Window L2,1.095699,5.0
3,BinSeg L2,1.081824,6.0
11,BottomUp Cosine,1.076064,7.0
28,Window RBF,1.071170,8.0
17,BottomUp RBF,1.061071,9.0
8,BinSeg Rank,1.054931,10.0


In [28]:
arr_ranking_df = compute_arr_ranking(metrics_df, acc_d=0.25, metric='Avg_WAPE')
arr_ranking_df

,ap,arr,rank
10,BottomUp Clinear,1.413496,1.0
16,BottomUp Normal,1.201670,2.0
4,BinSeg Linear,1.194264,3.0
20,Window AR,1.188621,4.0
13,BottomUp L2,1.146166,5.0
23,Window L1,1.135778,6.0
6,BinSeg Normal,1.133704,7.0
24,Window L2,1.124847,8.0
2,BinSeg L1,1.099485,9.0
22,Window Cosine,1.085268,10.0


In [29]:
arr_ranking_df = compute_arr_ranking(metrics_df, acc_d=0.5, metric='Avg_WAPE')
arr_ranking_df

,ap,arr,rank
16,BottomUp Normal,1.928995,1.0
4,BinSeg Linear,1.905947,2.0
6,BinSeg Normal,1.666540,3.0
14,BottomUp Linear,1.592883,4.0
20,Window AR,1.522546,5.0
26,Window Mahalanobis,1.495836,6.0
27,Window Normal,1.436145,7.0
29,Window Rank,1.428054,8.0
10,BottomUp Clinear,1.339217,9.0
13,BottomUp L2,1.279666,10.0


In [30]:
arr_ranking_df = compute_arr_ranking(metrics_df, acc_d=0.75, metric='Avg_WAPE')
arr_ranking_df

,ap,arr,rank
25,Window Linear,2.355900,1.0
20,Window AR,2.257975,2.0
26,Window Mahalanobis,1.649058,3.0
29,Window Rank,1.548412,4.0
27,Window Normal,1.528784,5.0
14,BottomUp Linear,1.426545,6.0
4,BinSeg Linear,1.388778,7.0
16,BottomUp Normal,1.350469,8.0
13,BottomUp L2,1.226594,9.0
12,BottomUp L1,1.210627,10.0


## MSE

In [31]:
arr_ranking_df = compute_arr_ranking(metrics_df, acc_d=0.05, metric='Avg_MSE')
arr_ranking_df

,ap,arr,rank
22,Window Cosine,1.229793,1.0
2,BinSeg L1,1.157935,2.0
11,BottomUp Cosine,1.156945,3.0
3,BinSeg L2,1.154163,4.0
17,BottomUp RBF,1.145061,5.0
13,BottomUp L2,1.122284,6.0
23,Window L1,1.120437,7.0
8,BinSeg Rank,1.116800,8.0
9,BottomUp AR,1.109661,9.0
5,BinSeg Mahalanobis,1.106599,10.0


In [32]:
arr_ranking_df = compute_arr_ranking(metrics_df, acc_d=0.1, metric='Avg_MSE')
arr_ranking_df

,ap,arr,rank
22,Window Cosine,1.210121,1.0
2,BinSeg L1,1.152035,2.0
11,BottomUp Cosine,1.144574,3.0
17,BottomUp RBF,1.135137,4.0
3,BinSeg L2,1.133990,5.0
13,BottomUp L2,1.126388,6.0
23,Window L1,1.114693,7.0
9,BottomUp AR,1.109294,8.0
8,BinSeg Rank,1.106872,9.0
24,Window L2,1.092895,10.0


In [33]:
arr_ranking_df = compute_arr_ranking(metrics_df, acc_d=0.25, metric='Avg_MSE')
arr_ranking_df

,ap,arr,rank
10,BottomUp Clinear,1.288604,1.0
16,BottomUp Normal,1.250021,2.0
4,BinSeg Linear,1.222846,3.0
20,Window AR,1.196794,4.0
13,BottomUp L2,1.171144,5.0
22,Window Cosine,1.169192,6.0
2,BinSeg L1,1.151702,7.0
9,BottomUp AR,1.138961,8.0
23,Window L1,1.135805,9.0
6,BinSeg Normal,1.128960,10.0


In [34]:
arr_ranking_df = compute_arr_ranking(metrics_df, acc_d=0.5, metric='Avg_MSE')
arr_ranking_df

,ap,arr,rank
16,BottomUp Normal,1.982866,1.0
4,BinSeg Linear,1.932816,2.0
6,BinSeg Normal,1.657665,3.0
20,Window AR,1.530123,4.0
27,Window Normal,1.452977,5.0
14,BottomUp Linear,1.445347,6.0
13,BottomUp L2,1.305157,7.0
26,Window Mahalanobis,1.298354,8.0
29,Window Rank,1.241292,9.0
9,BottomUp AR,1.230023,10.0


In [35]:
arr_ranking_df = compute_arr_ranking(metrics_df, acc_d=0.75, metric='Avg_MSE')
arr_ranking_df

,ap,arr,rank
20,Window AR,2.243333,1.0
25,Window Linear,2.049930,2.0
27,Window Normal,1.534170,3.0
4,BinSeg Linear,1.461170,4.0
16,BottomUp Normal,1.450539,5.0
26,Window Mahalanobis,1.436404,6.0
29,Window Rank,1.355648,7.0
14,BottomUp Linear,1.258289,8.0
13,BottomUp L2,1.254131,9.0
12,BottomUp L1,1.242127,10.0


## RMSE

In [36]:
arr_ranking_df = compute_arr_ranking(metrics_df, acc_d=0.05, metric='Avg_RMSE')
arr_ranking_df

,ap,arr,rank
22,Window Cosine,1.107250,1.0
2,BinSeg L1,1.093711,2.0
23,Window L1,1.090721,3.0
13,BottomUp L2,1.083150,4.0
11,BottomUp Cosine,1.079454,5.0
3,BinSeg L2,1.079183,6.0
24,Window L2,1.074233,7.0
28,Window RBF,1.066927,8.0
17,BottomUp RBF,1.064625,9.0
8,BinSeg Rank,1.060444,10.0


In [37]:
arr_ranking_df = compute_arr_ranking(metrics_df, acc_d=0.1, metric='Avg_RMSE')
arr_ranking_df

,ap,arr,rank
22,Window Cosine,1.090678,1.0
2,BinSeg L1,1.089347,2.0
13,BottomUp L2,1.088372,3.0
23,Window L1,1.086379,4.0
24,Window L2,1.073382,5.0
11,BottomUp Cosine,1.069059,6.0
3,BinSeg L2,1.061445,7.0
28,Window RBF,1.057903,8.0
17,BottomUp RBF,1.056544,9.0
8,BinSeg Rank,1.052153,10.0


In [38]:
arr_ranking_df = compute_arr_ranking(metrics_df, acc_d=0.25, metric='Avg_RMSE')
arr_ranking_df

,ap,arr,rank
10,BottomUp Clinear,1.477739,1.0
16,BottomUp Normal,1.222424,2.0
4,BinSeg Linear,1.213139,3.0
20,Window AR,1.206803,4.0
6,BinSeg Normal,1.157513,5.0
13,BottomUp L2,1.136311,6.0
14,BottomUp Linear,1.119730,7.0
23,Window L1,1.111830,8.0
24,Window L2,1.103206,9.0
2,BinSeg L1,1.092608,10.0


In [39]:
arr_ranking_df = compute_arr_ranking(metrics_df, acc_d=0.5, metric='Avg_RMSE')
arr_ranking_df

,ap,arr,rank
16,BottomUp Normal,1.964284,1.0
4,BinSeg Linear,1.939506,2.0
6,BinSeg Normal,1.695766,3.0
14,BottomUp Linear,1.627763,4.0
20,Window AR,1.547838,5.0
26,Window Mahalanobis,1.488498,6.0
27,Window Normal,1.464588,7.0
29,Window Rank,1.440433,8.0
10,BottomUp Clinear,1.407789,9.0
13,BottomUp L2,1.271833,10.0


In [40]:
arr_ranking_df = compute_arr_ranking(metrics_df, acc_d=0.75, metric='Avg_RMSE')
arr_ranking_df

,ap,arr,rank
25,Window Linear,2.448957,1.0
20,Window AR,2.303796,2.0
26,Window Mahalanobis,1.642558,3.0
29,Window Rank,1.564607,4.0
27,Window Normal,1.560399,5.0
14,BottomUp Linear,1.483014,6.0
4,BinSeg Linear,1.384828,7.0
16,BottomUp Normal,1.348567,8.0
12,BottomUp L1,1.220818,9.0
13,BottomUp L2,1.216689,10.0
